In [1]:


!pip install gref4hsi==0.2.5

import sys
import os

# Uncomment this block if developing on the gref4hsi
"""
!pip uninstall gref4hsi
home_path = "/home/fc-3auid-3af522edce-2ddb4d-2d4b8c-2d8500-2df5376270c3e0"
module_path = os.path.join(home_path, "gitprojects/gref4hsi")  # Replace with the actual path
sys.path.append(module_path)
import gref4hsi # Ensure that module is installed correctly, will throw error otherwize
"""
# Install py6s using conda (assuming mamba is a conda alias)
try:
    import Py6S
except ImportError:
    !mamba install -y py6s

!pip install rad4sea==0.0.3

In [2]:
print(sys.path)

['/home/fc-3auid-3af522edce-2ddb4d-2d4b8c-2d8500-2df5376270c3e0/gitprojects/seabeepy/notebooks', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages']


In [3]:


# Standard python library
import configparser
import sys
import os
import argparse
from collections import namedtuple

# Local resources
from gref4hsi.scripts import georeference
from gref4hsi.scripts import orthorectification
from gref4hsi.scripts import coregistration
from gref4hsi.utils import parsing_utils, specim_parsing_utils
from gref4hsi.utils import visualize
from gref4hsi.utils.config_utils import prepend_data_dir_to_relative_paths
from gref4hsi.utils.config_utils import customize_config

# Third party
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
# From seabeepy/notebooks/flight_runner
import datetime as dt
import os
from pathlib import Path

from seabeepy.config import SETTINGS
from pyodm import Node
from subprocess import CalledProcessError

import seabeepy as sb
import rad4sea

In [5]:


# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)



In [6]:
# Parent directories containing flight folders to process
base_dirs = [
    r"/home/notebook/shared-seabee-ns9879k/ntnu",
]

# Directory for temporary files
temp_dir = r"/home/notebook/cogs"

In [7]:
# Run info
run_date = dt.datetime.today()
print(f"Processing started: {run_date}")

Processing started: 2024-09-06 11:12:09.565433


In [8]:
# Get all potential mission folders for NodeODM
# (i.e. folders containing a 'config.seabee.yaml' and an 'capture' subdirectory, but NOT a 'processed' directory)
mission_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "capture")
    and not sb.ortho.check_subdir_exists(f.parent, "processed")
]

In [9]:
mission_list

[PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-09-01-Runde/runde_2022090114225_ntnu_hyperspectral_74m')]

In [10]:
# Establish the ancillary data paths, copy to local work space and 
from pathlib import Path
import shutil

geoid_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/geoids/no_kv_HREF2018A_NN2000_EUREF89.tif')
config_template_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim.ini')
lab_calibration_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations')

# Local path geoid
geoid_path = os.path.join(temp_dir, "no_kv_HREF2018A_NN2000_EUREF89.tif")
try:
    shutil.copyfile(geoid_path_minio, geoid_path)
except FileExistsError:
    pass

# Local path for configuration file
config_template_path = os.path.join(temp_dir, "config_template_path_specim.ini")
try:
    shutil.copyfile(config_template_path_minio, config_template_path)
except FileExistsError:
    pass

# Local path for lab calibration
lab_calibration_path = os.path.join(temp_dir, "lab-calibration")
try:
    shutil.copytree(lab_calibration_path_minio, lab_calibration_path)
except FileExistsError:
    pass

In [11]:
%load_ext autoreload
%autoreload 2

import shutil

import process_hyperspectral

# Convenient for development, effectively avoiding kernel restart
import importlib
importlib.reload(process_hyperspectral)

# Process missions in the list
for specim_mission_folder_minio in mission_list:

    mission_name = specim_mission_folder_minio.name
    print(f"\n################\nProcessing: {mission_name}")

    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)

    # Copy to local fs if not there already
    try:
        print('Copying file')
        shutil.copytree(specim_mission_folder_minio, specim_mission_folder)
    except FileExistsError:
        pass

    # The config file is read
    config_yaml = os.path.join(specim_mission_folder, "config.seabee.yaml")

    # Specim processing to get georeferenced radiance data
    # Setting fast_mode to true can be smart for development purposes as it creates a coarse image
    process_hyperspectral.main(str(config_yaml), 
                        str(specim_mission_folder), 
                        geoid_path, 
                        config_template_path, 
                        lab_calibration_path,
                        fast_mode = False)




################
Processing: runde_2022090114225_ntnu_hyperspectral_74m
Copying file
DEM folder does not exist so Geoid is used as terrain instead
Coregistration is not done, as there was no reference orthomosaic


/opt/conda/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
Computing 2D Triangulation: 100%|██████████[00:00<00:00]



################ Georeferencing with a Brake: ################
runde_2022090114225_ntnu_hyperspectral_74m_transectnr_0_chunknr_0.h5
Georeferencing file 1/8, progress is 0.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2948713.57667742  291573.47080665 5629184.21874757]
runde_2022090114225_ntnu_hyperspectral_74m_transectnr_1_chunknr_0.h5
Georeferencing file 2/8, progress is 12.5 %
All rays were successfully intersected with trimesh
Mesh translation is [2948713.57667742  291573.47080665 5629184.21874757]
runde_2022090114225_ntnu_hyperspectral_74m_transectnr_2_chunknr_0.h5
Georeferencing file 3/8, progress is 25.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2948713.57667742  291573.47080665 5629184.21874757]
runde_2022090114225_ntnu_hyperspectral_74m_transectnr_3_chunknr_0.h5
Georeferencing file 4/8, progress is 37.5 %
All rays were successfully intersected with trimesh
Mesh translation is [2948713.57667742  291573.47080665 562

## 3. Convert radiance data into reflectance by division with simulated spectrum from Py6S

In [17]:
# Raster and ancillary (
import rasterio
from rasterio.plot import show
import spectral as sp
import shutil

import glob

# Uncomment if you are doing module development
"""
module_path = os.path.join('/home/notebook/', 'gitprojects/rad4sea/')
if module_path not in sys.path:
    sys.path.append(module_path)
"""

import rad4sea
import rad2refl

import importlib
importlib.reload(rad4sea)
importlib.reload(rad2refl)

# The radiance multiplier for the specim afx10 relative to unit of W/(m^2*sr*nm)
radiance_multiplier_specim = (1 / 1000) #(mW/cm^2*sr*um)*1000.0000 ->(mW/cm^2*sr*um)
radiance_multiplier_specim *= (1e-3 / 1e-4) #(mW/cm^2*sr*um) -> (W/m^2*sr*um)
radiance_multiplier_specim *= (1 / 1e3) # (W/m^2*sr*um) -> (W/m^2*sr*nm)



for specim_mission_folder_minio in mission_list:

    # Select a particular transect datacube:
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)

    print(specim_mission_folder)

    # Where the cube and anc data are
    
    if os.path.exists( os.path.join(specim_mission_folder, "processed/Output/GIS/") ):
        cube_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/HSIDatacubes/")
        anc_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/AncillaryData/")
    else:
        # If already processed and reflectance processing is to be conducted with overwriting
        cube_folder = os.path.join(specim_mission_folder, "processed/cubes")
        anc_folder = os.path.join(specim_mission_folder, "processed/ancillary")
        
    # Make copies of radiance data for manipulation
    for filename in os.listdir(cube_folder):
        if filename.lower().endswith(".img") or filename.lower().endswith(".hdr"):  # Check for lowercase extension
            base, ext = os.path.splitext(filename)  # Separate base name and extension

            if base.split('_')[-1] == 'reflectance':
                pass
            else:
                new_filename = f"{base}_reflectance{ext}"  # Construct new filename with suffix
                source_file = os.path.join(cube_folder, filename)
                destination_file = os.path.join(cube_folder, new_filename)
                # Copy the file
                
                shutil.copyfile(source_file, destination_file)

    # We only modulate the copy of the data (with _reflectance suffix)
    cube_hdr_list = glob.glob(cube_folder + '/*_reflectance.hdr')

    # Simulates downwelling irradiance and computes remote sensing reflectance 
    rad2refl.main(anc_folder = anc_folder, cube_list_refl = cube_hdr_list, cube_folder = cube_folder, radiance_multiplier=radiance_multiplier_specim)



/home/notebook/cogs/runde_2022090114225_ntnu_hyperspectral_74m
2022-09-01 12:27:25
2022-09-01
5.657378112617663, 62.392742604072794, 0.07437044209753306


100%|██████████| 1420/1420 [03:56<00:00,  6.01it/s]


2022-09-01 12:30:02
2022-09-01
5.654815846304763, 62.39312721035097, 0.07500328768056067


100%|██████████| 1420/1420 [03:56<00:00,  6.00it/s]


2022-09-01 12:24:47
2022-09-01
5.658855392396938, 62.392865505817824, 0.07520167062116188


...

100%|██████████| 1420/1420 [03:55<00:00,  6.02it/s]


2022-09-01 12:32:46
2022-09-01
5.6570636594042325, 62.391510489484034, 0.07367331815978945


100%|██████████| 1420/1420 [03:57<00:00,  5.99it/s]


2022-09-01 12:22:45
2022-09-01
5.656197852432262, 62.394376632549886, 0.0794675945517747


100%|██████████| 1420/1420 [03:56<00:00,  6.01it/s]


In [18]:
mission_list


[PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-09-01-Runde/runde_2022090114225_ntnu_hyperspectral_74m')]

In [19]:
"""A simple demonstration of the data for a transect showing radiance, reflectance and an RGB composite"""

# Try to visualize 
try:
    specim_mission_folder_minio = mission_list[0]
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
    cube_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/HSIDatacubes/")
    anc_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/AncillaryData/")

    cube_hdr_list = glob.glob(cube_folder + '/*_reflectance.hdr')
    import matplotlib.pyplot as plt

    # Example plotting of radiance data
    refl_image_obj = sp.io.envi.open(cube_hdr_list[0])


    band_im_refl = refl_image_obj[:, :, 100]
    nodata = float(refl_image_obj.metadata['data ignore value'])
    wl = np.array(refl_image_obj.metadata["wavelengths"]).astype(np.float64)

    valids = (band_im_refl != nodata).squeeze()

    n_rows, n_cols, _ = band_im_refl.shape

    row_indices = np.repeat(np.arange(n_rows).reshape((-1, 1)), n_cols, axis = 1)
    col_indices = np.repeat(np.arange(n_cols).reshape((1, -1)), n_rows, axis = 0)
    #print(row_indices.shape)
    row_valids = row_indices[valids]
    col_valids = col_indices[valids]

    n_valids = row_valids.size
    spec_idx_example = int(n_valids/2) # Some random spectrum in the image. Index can be from 0 to n_valids


    print(cube_hdr_list[0])
    plt.plot(wl, refl_image_obj[row_valids[spec_idx_example], col_valids[spec_idx_example], :].flatten())
    plt.ylabel('Remote sensing reflectance [1/sr]')
    plt.xlabel('Nanometers [nm]')
    plt.show()

    rad_image_obj = sp.io.envi.open('_'.join(cube_hdr_list[0].split('_')[0:-1]) + '.hdr') 
    rad_specim_example = rad_image_obj[row_valids[spec_idx_example], col_valids[spec_idx_example], :].flatten()
    plt.plot(wl, rad_specim_example*radiance_multiplier_specim)
    plt.ylabel('Radiance [mW/(m$^2$ sr nm)]')
    plt.xlabel('Nanometers [nm]')
    plt.show()

    pad = 10
    rows_zoom = np.arange(row_valids[spec_idx_example]-pad, row_valids[spec_idx_example]+pad)
    cols_zoom = np.arange(col_valids[spec_idx_example]-pad, col_valids[spec_idx_example]+pad)



    RGB = refl_image_obj[:, :, [73, 50, 24]]

    RGB.shape

    # Emulates a gamma stretch to bring out contrast in dark areas
    plt.imshow((RGB/RGB.max())**0.44, vmin = 0)
    plt.scatter(row_valids[spec_idx_example], col_valids[spec_idx_example], label = 'Sample spectrum')
    plt.legend()
    plt.show()
except:
    pass    




## 4. Transfer to MinIO and Publish to GeoNode

### 4.1 Convert RGB composites into one one mosaic (no fancy stretching or similar yet)


In [20]:
from osgeo import gdal
import rasterio as rio

def merge_rasters(raster_file_list, output_file):
    """Merges the geotif files in a directory into one geoTIF"""
    ds_lst = list()
    for raster in raster_file_list:
        ds = gdal.Warp('', raster, format='vrt')
        ds_lst.append(ds)
    dataset = gdal.BuildVRT('', ds_lst)
    ds1 = gdal.Translate(output_file, dataset)
    del ds1  
    del dataset
    # Add band info if not there yet
    with rio.open(output_file, "r+") as src:
        if any(val is None for val in src.descriptions):
            src.descriptions = ("red", "green", "blue")
    
    return output_file

In [21]:
import numpy as np

def gamma_transform(data, gamma=0.44, min_prc = 2, max_prc = 98):
    """Applies gamma transformation to data.

    Args:
        data: Input 3 band radiance data.
        gamma: Gamma value for the transformation.

    Returns:
        Gamma-transformed 8-bit data array.
    """
    
    data_min_prc = np.percentile(data, min_prc)
    
    data_max_prc = np.percentile(data, max_prc)
    
    norm_data = (data-data.min())/(data.max()-data.min())
    
    # Mapping that enhances dark areas if gamma < 1
    gamma_data = (norm_data**gamma) * 255
    
    # Stretch data in case there are some very bright areas in image
    
    data_min_prc = np.percentile(gamma_data, min_prc)
    
    data_max_prc = np.percentile(gamma_data, max_prc)
    
    
    undersaturated = gamma_data <= data_min_prc
    oversaturated = gamma_data >= data_max_prc
    
    
    # Rest of data is stretched
    gamma_data = np.ceil(( (gamma_data-data_min_prc)/(data_max_prc-data_min_prc) ) * 255 )
    
    
    
    gamma_data[undersaturated] = 1
    gamma_data[oversaturated] = 255
    
    
    # Cast to 8 bit
    gamma_data = gamma_data.astype(np.uint8)
    
    return gamma_data

def apply_gamma_transform(raster_file, gamma):
    """Transforms the RGB raster to enhance color"""
    with rio.open(raster_file, "r+") as src:
        profile = src.profile
        count = src.count
        
        # Update profile for 8-bit unsigned integer data type
        kwargs = profile
        kwargs.update(
            dtype=rasterio.uint8,
            nodata=0)
        

        # Create a new dataset for writing
        with rasterio.open(raster_file, 'w', **kwargs) as dst:
            for i in range(1, count + 1):
                # Read the band data
                band_data = src.read(i)
                
                gamma_data = np.zeros(band_data.shape, dtype = np.uint8)
                
                # Where the data is
                data_mask = band_data != src.nodata
                
                # Apply gamma transformation
                gamma_data[data_mask] = gamma_transform(band_data[data_mask], gamma)
                
                print(dst.nodata)

                # Write the gamma-transformed data back to the band
                dst.write(gamma_data, i)
                
            if any(val is None for val in dst.descriptions):
                dst.descriptions = ("red", "green", "blue")

### 4.2 Copy processed data to specim_mission_folder_minio/processed

In [23]:
import glob
import rasterio

for specim_mission_folder_minio in mission_list:
    # Select a particular transect datacube:
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
    
    # Where the cube and anc data are
    #cube_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/HSIDatacubes/")
    #anc_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/AncillaryData/")
    
    # Processed Cubes MINIO
    cubes_path_minio = os.path.join(specim_mission_folder_minio, 'processed/cubes')

    # Processed ancillary cubes MINIO
    anc_cube_path_minio = os.path.join(specim_mission_folder_minio, 'processed/ancillary')

    # Processed composites MINIO are now stored under composites_path
    # And should be moved to 
    composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')
    
    # Where is the processed data
    cube_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/HSIDatacubes/")
    anc_folder = os.path.join(specim_mission_folder, "processed/Output/GIS/AncillaryData/")
    composites_path = os.path.join(specim_mission_folder, 'processed/Output/GIS/RGBComposites')
    
    # Processed composites are now stored under
    
    # And should be moved to 
    composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')
    
    composite_list = glob.glob(composites_path + '/*.tif')
    
    # The combined version is to be deployed on GEONODE
    combined_composite_filename = os.path.join(composites_path, "combined.tif")
    
    merge_rasters(composite_list, combined_composite_filename)
    
    apply_gamma_transform(combined_composite_filename, gamma = 0.44)
    
    # Then we transfer the results to MinIO
    # Processed datacubes are now stored under
    cubes_path = cube_folder
    cubes_path_minio = os.path.join(specim_mission_folder_minio, 'processed/cubes')

    # Processed ancillary cubes are now stored under
    anc_cube_path = anc_folder
    anc_cube_path_minio = os.path.join(specim_mission_folder_minio, 'processed/ancillary')

    # Processed composites are now stored under composites_path
    # And should be moved to 
    composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')

    # Move datacube, composites and ancillary data to persistant storage on Minio
    # Cubes (radiance and reflectance)
    
    try:
        sb.storage.copy_folder(src_fold = cubes_path, dst_fold = cubes_path_minio, client=minio_client, containing_folder = False, overwrite = False)
    
        # Composite data (One per transect and a combined for visualization)
        sb.storage.copy_folder(src_fold = composites_path, dst_fold = composites_path_minio, client = minio_client, containing_folder = False, overwrite = False)
        #sb.storage.copy_file(combined_composite_filename, os.path.join(composites_path_minio,'combined.tif'), client = minio_client, overwrite = True)
    
        # Ancillary data for further processing
        sb.storage.copy_folder(src_fold = anc_cube_path, dst_fold = anc_cube_path_minio, client = minio_client, containing_folder = False, overwrite = False)
    except:
        processed_path = os.path.join(specim_mission_folder_minio, 'processed')
        print(f'The folder {processed_path} already exists, no copying is done.')
 


0.0
0.0
0.0


### 4.3 Publish data to Geonode/Geoserver

In [24]:
# Identify datasets for publishing. Folders must contain either an ODM or Pix4D
# original orthophoto (not both) and must not contain a COG named f'{layer_name}.tif'.
# Folders must also have 'config.seabee.yaml' files where 'publish' is True
publish_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "capture")
    and sb.ortho.check_subdir_exists(f.parent, "processed")
    and not sb.ortho.check_subdir_exists(f.parent, "orthophoto")
    and sb.ortho.parse_config(f.parent)["publish"]
]

print("The following missions are publishable to GeoNode:")
print(publish_list)

The following missions are publishable to GeoNode:
[PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-09-01-Runde/runde_2022090114225_ntnu_hyperspectral_74m')]


In [25]:
import rasterio as rio
import yaml
import shutil
import os
# Publish publishables (equivalent to the same section in flights_runner.ipynb without the ODM/Pix4D stuff which does not apply here)
for mission_fold in publish_list:
    mission_name = mission_fold.name
    

    # Orthophoto is termed "combined.tif"
    ortho_path = os.path.join(
        mission_fold, "processed/composites", "combined.tif"
    )
    
    config_yaml_minio = os.path.join(
        mission_fold, "config.seabee.yaml"
    )
    
    specim_mission_folder = os.path.join(temp_dir, mission_name)
    
    try:
        h5_dir = os.path.join(specim_mission_folder, 'processed/Input/H5')
        nfiles = len(os.listdir(h5_dir))
    except:
        
        continue
    
    print(f"\n################\nProcessing: {mission_name}")
    print("Preparing orthophoto for publishing.")
        
        
        
        
    
    print(nfiles)
    
    # Prior to updating metadata we need to set the "nfiles" entry in config.seabee.yaml
    
    config_yaml = os.path.join(specim_mission_folder, "config.seabee.yaml")
    
    # Add entry nfiles
    with open(config_yaml, 'r+') as file:  
        config_data = yaml.safe_load(file)
        config_data['nfiles'] = nfiles
        config_data['organisation'] = 'NTNU'
        
        # After modifying the data, write it back to the file
        yaml.safe_dump(config_data, file)
        
    with open(config_yaml, 'r') as file:  
        config_data = yaml.safe_load(file)
        print(config_data['nfiles'])
        print(config_data['creator_name'])
    
    # Overwrite minio config version
    sb.storage.copy_file(config_yaml, config_yaml_minio, client = minio_client, overwrite = True)
    
    
    
    # Standardise and save locally
    layer_name = sb.ortho.get_layer_name(mission_fold)
    temp_path = os.path.join(temp_dir, layer_name + ".tif")
    
    try:
        sb.geo.standardise_orthophoto(
            ortho_path,
            temp_path)
    except CalledProcessError as e:
        print(f"Failed to standardise {ortho_path}.")
        print(e)
        continue

    # Copy to MinIO and delete local version
    stan_path = os.path.join(mission_fold, "orthophoto", layer_name + ".tif")
    sb.storage.copy_file(temp_path, stan_path, minio_client, overwrite=True)
    os.remove(temp_path)

    print("Uploading to GeoServer.")

    sb.geo.upload_raster_to_geoserver(
        stan_path,
        SETTINGS.GEOSERVER_USER,
        SETTINGS.GEOSERVER_PASSWORD,
        workspace="geonode",
    )

    print("Publishing to GeoNode.")

    sb.geo.publish_to_geonode(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        workspace="geonode",
    )
    
    

    print("Updating metadata.")
    date = sb.ortho.parse_mission_data(mission_fold, parse_date=True)[2]
    abstract = sb.geo.get_html_abstract(str(mission_fold))
    metadata = {
        "abstract": abstract,
        "date": date.isoformat(),
        "date_type": "creation",
        "attribution": "SeaBee",
    }
    sb.geo.update_geonode_metadata(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        metadata,
    )
    print(layer_name)


################
Processing: runde_2022090114225_ntnu_hyperspectral_74m
Preparing orthophoto for publishing.
8
8
Havard Snefjella Lovas
Input file size is 709, 714
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GeoServer.
Publishing to GeoNode.
Updating metadata.
runde_more-romsdal_202209011425_HSI_74m


In [ ]:
sb.geo.infer_geotiff_metadata(ortho_path)

### 4.4 Delete local data

In [26]:
"""Delete the local data. This should only be done once everything is well in place"""
for specim_mission_folder_minio in mission_list:
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
    shutil.rmtree(specim_mission_folder)